<a href="https://colab.research.google.com/github/NandiniLReddy/Llama-3.2/blob/main/llama3_2vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 3.2 deployed by Nandini Lokesh Reddy

In [ ]:
from huggingface_hub import login
login(token="HUGGING FACE TOKEN")

In [ ]:
pip install transformers

## Usecase: Text generation

In [ ]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
# Specify the device to use GPU (device 0 by default)
device = 0 if torch.cuda.is_available() else -1  # -1 indicates CPU

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device=device  # Use GPU if available
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

outputs = pipe(
    messages,
    max_new_tokens=256
)

print(outputs[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': "Arrrr, ye be askin' about meself, eh? Alright then, matey! I be Captain Blackbeak Betty, the most feared and infamous pirate to ever sail the Seven Seas. Me and me crew o' scurvy dogs have been plunderin' and pillagin' fer years, bringin' riches and glory to our names. Me trusty parrot, Polly, be me right-hand scallywag, and me loyal cutlass, Cutlass Cutie, be me best mate. We be sailin' the Caribbean, searchin' fer hidden treasure and fightin' off any landlubbers who get in our way. So hoist the sails and set course fer adventure with me and me crew, or walk the plank, matey!"}]


## Usecase: Topic Generation

In [ ]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
# Use GPU if available, otherwise CPU
device = 0 if torch.cuda.is_available() else -1

# Initialize the text-generation pipeline with the specified model
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device=device
)

# Example of a user-provided context
user_input = """L&D practitioners spend a lot of time and effort creating excellent leadership development programs, yet year after year companies report leadership as lacking.
The main reason for this? Once the participants leave the training program, it’s difficult to hold them accountable and ensure the learnings are applied at work.
Leveraging technologies like AI, NLU and NLP can help address this by serving as ongoing mentors to guide leaders to foster organisations with engaged employees.."""

# System instructs the model to generate topics based on the context
messages = [
    {"role": "system", "content": "Generate a list of topics related to the following context:"},
    {"role": "user", "content": user_input}
]

# Generate output with the specified number of new tokens
outputs = pipe(
    messages,
    max_new_tokens=50  # Adjust token limit based on typical length needed for topic lists
)

# Print the generated topics
print(outputs[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': 'Generate a list of topics related to the following context:'}, {'role': 'user', 'content': 'L&D practitioners spend a lot of time and effort creating excellent leadership development programs, yet year after year companies report leadership as lacking.\nThe main reason for this? Once the participants leave the training program, it’s difficult to hold them accountable and ensure the learnings are applied at work.\nLeveraging technologies like AI, NLU and NLP can help address this by serving as ongoing mentors to guide leaders to foster organisations with engaged employees..'}, {'role': 'assistant', 'content': "Here's a list of topics related to the given context:\n\n**Leadership Development Program Challenges**\n\n1. **Lack of Leadership Skills**: Identifying and developing effective leadership skills in employees.\n2. **Retention and Engagement**: Keeping employees engaged and motivated"}]


## 11B Vision model: image description

In [ ]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
)
model = model.to(device)

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Load an image from the web
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Create a message list that includes an image and a text prompt
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "If I had to write a haiku for this one, it would be: "}
    ]}
]

# Prepare inputs using the processor
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(image, input_text, return_tensors="pt").to(device)

# Generate output from the model
output = model.generate(**inputs, max_new_tokens=30)
print(processor.decode(output[0]))


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>If I had to write a haiku for this one, it would be: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here is a haiku for the image:

A rabbit in a coat
Stands on a dirt road, smiling
Springtime in the air<|eot_id|>
